In [1]:
import numpy as np

In [5]:
class Simplex:

    def __init__(self, conditii, problema):
        self.conditii = conditii
        self.problema = problema

    def to_primal(self):
        self.problema = (np.array(self.problema) * (-1)).tolist()
    
    def to_dual(self):
        conditii_np = np.array(self.conditii)
        problema_np = np.array(self.problema)

        conditii_noi = (conditii_np[:, :-2].astype('float64') * (-1)).T.tolist()
        for i, inegalitate in enumerate(conditii_np[:, -2]):
            conditii_noi[i].append(inegalitate)
            
        for i, comp in enumerate(self.problema):
            conditii_noi[i].append(comp * (-1))
            
        problema_noua = conditii_np[:, -1].tolist()

        self.conditii = conditii_noi
        self.problema = problema_noua
    
    def to_standard(self):
        self.A = []
        self.nr_s = 0

        for conditie in self.conditii:
            if conditie[-2] == '<=' or conditie[-2] == '>=':
                self.nr_s += 1
            
        crt_s = 0
        for conditie in self.conditii:
            if conditie[-2] == '=':
                self.A.append(conditie[:-2] + [0] * self.nr_s + [conditie[-1]])
            elif conditie[-2] == '<=':
                self.A.append(conditie[:-2] + [0] * crt_s + [1] + [0] * (self.nr_s - crt_s - 1) + [conditie[-1]])
                crt_s += 1
            elif conditie[-2] == '>=':
                self.A.append(conditie[:-2] + [0] * crt_s + [-1] + [0] * (self.nr_s - crt_s - 1) + [conditie[-1]])
                crt_s += 1
            
        self.A.append(self.problema + [0] * (self.nr_s + 1))
        self.A = np.array(self.A, dtype='float64')
        self.nr_x = self.A.shape[1] - self.nr_s - 1
    

    def pivot(self, rand, coloana):
        self.A[rand] /= self.A[rand, coloana]
        
        for i in range(len(self.A)):
            if i != rand:
                self.A[i] -= self.A[i, coloana] * self.A[rand]
    
    def solve_primal(self):
        while True:
            print(self.A)
            coloana_pivot = -1
            for i, val in enumerate(self.A[-1]):
                if val < 0:
                    coloana_pivot = i
                    break
            
            if coloana_pivot == -1:
                break

            rand_pivot = -1
            raport_minim = np.inf
            for i in range(len(self.A) - 1):
                st = self.A[i, coloana_pivot]
                dr = self.A[i, -1]
                raport = dr / st
                if raport < raport_minim:
                    raport_minim = raport
                    rand_pivot = i

            if rand_pivot == -1:
                print('z = inf')
                break

            self.pivot(rand_pivot, coloana_pivot)
        
        print(f'A final:\n{self.A}', end='\n\n')

        print(f'z_opt = {self.A[-1][-1]}', end='\n\n')

        for x in range(self.nr_x):
            print(f'x{x + 1} = {self.A[x, -1]}')

    def solve_dual(self):
        while True:
            coloana_pivot = -1
            for i, val in enumerate(self.A[-1]):
                if val > 0:
                    coloana_pivot = i
                    break
            
            if coloana_pivot == -1 or coloana_pivot >= self.nr_x:
                break

            rand_pivot = -1
            raport_minim = np.inf
            for i in range(len(self.A) - 1):
                st = self.A[i, coloana_pivot]
                dr = self.A[i, -1]
                raport = dr / st
                if raport < raport_minim:
                    raport_minim = raport
                    rand_pivot = i

            if rand_pivot == -1:
                print('z = inf')
                break

            self.pivot(rand_pivot, coloana_pivot)
        
        print(f'A final:\n{self.A}', end='\n\n')

        print(f'w_opt = {self.A[-1][-1] * (-1)}', end='\n\n')

        for x in range(self.nr_x):
            print(f'y{x + 1} = {self.A[x, -1]}')

In [6]:
s = Simplex([
        [2, 0, -1, '<=', 2], 
        [1, 2, 2, '<=', 1],
        [2, 2, -1, '<=', 5]
    ],
        [2, 1, -1]
)

s.to_primal()
s.to_standard()
print(f'A:\n{s.A}', end='\n\n')
s.solve_primal()

A:
[[ 2.  0. -1.  1.  0.  0.  2.]
 [ 1.  2.  2.  0.  1.  0.  1.]
 [ 2.  2. -1.  0.  0.  1.  5.]
 [-2. -1.  1.  0.  0.  0.  0.]]

[[ 2.  0. -1.  1.  0.  0.  2.]
 [ 1.  2.  2.  0.  1.  0.  1.]
 [ 2.  2. -1.  0.  0.  1.  5.]
 [-2. -1.  1.  0.  0.  0.  0.]]
[[ 1.   0.  -0.5  0.5  0.   0.   1. ]
 [ 0.   2.   2.5 -0.5  1.   0.   0. ]
 [ 0.   2.   0.  -1.   0.   1.   3. ]
 [ 0.  -1.   0.   1.   0.   0.   2. ]]
[[ 1.    0.   -0.5   0.5   0.    0.    1.  ]
 [ 0.    1.    1.25 -0.25  0.5   0.    0.  ]
 [ 0.    0.   -2.5  -0.5  -1.    1.    3.  ]
 [ 0.    0.    1.25  0.75  0.5   0.    2.  ]]
A final:
[[ 1.    0.   -0.5   0.5   0.    0.    1.  ]
 [ 0.    1.    1.25 -0.25  0.5   0.    0.  ]
 [ 0.    0.   -2.5  -0.5  -1.    1.    3.  ]
 [ 0.    0.    1.25  0.75  0.5   0.    2.  ]]

z_opt = 2.0

x1 = 1.0
x2 = 0.0
x3 = 3.0
<ipython-input-5-cb683ecde273>:74: RuntimeWarning: divide by zero encountered in double_scalars
  raport = dr / st


In [237]:
s = Simplex([
        [2, 1, '<=', 12], 
        [2, 3, '<=', 16]
    ],
        [3, 2]
)

s.to_dual()
s.to_standard()
print(f'A:\n{s.A}', end='\n\n')
s.solve_dual()

A:
[[-2. -2.  1.  0. -3.]
 [-1. -3.  0.  1. -2.]
 [12. 16.  0.  0.  0.]]

A final:
[[  1.     0.    -0.75   0.5    1.25]
 [ -0.     1.     0.25  -0.5    0.25]
 [  0.     0.     5.     2.   -19.  ]]

w_opt = 19.0

y1 = 1.25
y2 = 0.25


In [ ]:
2 + 0 + 0